# Introduction

Please also upvote the original notebooks:

https://www.kaggle.com/code/oxzplvifi/pixel-travel-map

https://www.kaggle.com/code/crodoc/82409-improved-baseline-santa-2022

The two things new here:
- Removing the duplicates as post processing step
- Adding progress bar

This notebook uses a binary matrix in order to keep a record of each pixel that has been visited, called a "pixel travel map". During its travels, the arm is asked to follow three simple rules:
* If it is possible to go one step down, then ignore all other directions and go one step down. This is with the intention of avoiding "travel holes" in the map which could become costly to return to once we are far away.
* If it is not possible to go one step down, then consider all possible single-link and double-link motions and choose the one that (1) takes us to an unvisited pixel, and (2) has the least cost.
* If we become surrounded by visited pixels, then we travel to the nearest unvisited pixel no matter how many visited pixels we need to re-visit.

We start by loading the functions kindly shared by the organizers:
* https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2022

The path compression basically tries to do move as many links the same time as possible to save on reconfiguration cost.

In [1]:
from functools import reduce
from tqdm.notebook import tqdm

def get_position(config):
    return reduce(lambda p, q: (p[0] + q[0], p[1] + q[1]), config, (0, 0))

def compress_path(path):
    
    if len(path) > 2:
        
        new_path = []
        
        max_conf_dist = 1
        
        r = [[] for _ in range(len(path[0]))]
        
        for p in path:
            for i, c in enumerate(p):

                if len(r[i]) == 0 or r[i][-1] != c:
                    
                    if c not in r[i]:
                        r[i].append(c)
                    else:
                        r[i] = r[i][:r[i].index(c) + 1]
                        
                    assert r[i][-1] == c
        
        max_conf_dist = max([len(r_) for r_ in r])
        
        for i in range(max_conf_dist):
            
            new_conf = []
            
            for _, r_ in enumerate(r):
                
                if i < len(r_):
                    c_ = r_[i]
                else:
                    c_ = r_[-1]
                
                new_conf.append(c_)
            
            new_path.append(new_conf)
            
        return new_path
                               
        
    return path

def rotate_link(vector, direction):
    x, y = vector
    if direction == 1:  # counter-clockwise
        if y >= x and y > -x:
            x -= 1
        elif y > x and y <= -x:
            y -= 1
        elif y <= x and y < -x:
            x += 1
        else:
            y += 1
    elif direction == -1:  # clockwise
        if y > x and y >= -x:
            x += 1
        elif y >= x and y < -x:
            y += 1
        elif y < x and y <= -x:
            x -= 1
        else:
            y -= 1
    return (x, y)

def rotate(config, i, direction):
    config = config.copy()
    config[i] = rotate_link(config[i], direction)
    return config

def get_direction(u, v):
    """Returns the sign of the angle from u to v."""
    direction = np.sign(np.cross(u, v))
    if direction == 0 and np.dot(u, v) < 0:
        direction = 1
    return direction

def color_cost(from_position, to_position, image, color_scale=3.0):
    return np.abs(image[to_position] - image[from_position]).sum() * color_scale

def get_path_to_point(config, point):
    """Find a path of configurations to `point` starting at `config`."""
    path = [config]
    # Rotate each link, starting with the largest, until the point can
    # be reached by the remaining links. The last link must reach the
    # point itself.
    for i in range(len(config)):
        link = config[i]
        base = get_position(config[:i])
        relbase = (point[0] - base[0], point[1] - base[1])
        position = get_position(config[:i+1])
        relpos = (point[0] - position[0], point[1] - position[1])
        radius = reduce(lambda r, link: r + max(abs(link[0]), abs(link[1])), config[i+1:], 0)
        # Special case when next-to-last link lands on point.
        if radius == 1 and relpos == (0, 0):
            config = rotate(config, i, 1)
            if get_position(config) == point:  # Thanks @pgeiger
                path.append(config)
                break
            else:
                continue
        while np.max(np.abs(relpos)) > radius:
            direction = get_direction(link, relbase)
            config = rotate(config, i, direction)
            path.append(config)
            link = config[i]
            base = get_position(config[:i])
            relbase = (point[0] - base[0], point[1] - base[1])
            position = get_position(config[:i+1])
            relpos = (point[0] - position[0], point[1] - position[1])
            radius = reduce(lambda r, link: r + max(abs(link[0]), abs(link[1])), config[i+1:], 0)
    assert get_position(path[-1]) == point
    
    path = compress_path(path)
    
    return path

def get_path_to_configuration(from_config, to_config):
    path = [from_config]
    config = from_config.copy()
    while config != to_config:
        for i in range(len(config)):
            config = rotate(config, i, get_direction(config[i], to_config[i]))
        path.append(config)
    assert path[-1] == to_config
    
    path = compress_path(path)
    
    return path

def config_to_string(config):
    return ';'.join([' '.join(map(str, vector)) for vector in config])

Next, in order to simplify conversion of coordinates during our travels from Cartesian to Array and viceversa, we flip the X axis and transpose the X-Y axes of the image so that the conversion becomes:
* From Cartesian coordinates to Array indices we sum the image radius to the X and Y coordinates.
* From Array indices to Cartesian coordinates we subtract the image radius from the X and Y indices.

In [2]:
import numpy as np, pandas as pd

# Read image as a numpy array:
df_image = pd.read_csv('/kaggle/input/santa-2022/image.csv')
side = df_image.x.nunique()
radius = df_image.x.max()
image = df_image[['r','g','b']].values.reshape(side,side,-1)

# Flip X axis and transpose X-Y axes to simplify cartesian to array mapping:
image = image[::-1,:,:]
image = np.transpose(image, (1, 0, 2))

# Traveling

In more detail, the algorithm consists of the following simple steps:
1. Create a binary array containing a value of 1 for every pixel which has not been visited and a value of 0 for every pixel which has been visited.
2. Consider all single-link and double-link steps which end up in a place that we have not visited before, keeping a record of the step with the lowest cost. If it is possible to go one step down, then we go down and ignore all other options. If it is not possible to go down, then we go in the direction of least cost. We always push the arm to go down in order to avoid the creation of unvisited holes in the path, which will be costly to re-visit once we have moved far away.
3. If the arm becomes completely surrounded by visited sites ("stuck condition"), first we search for the nearest unvisited site, then we use the "get_path_to_point" function shared by the organizers in order to reach it in the shortest path.
4. Once every single pixel in the travel map has been completely visited, we ask our arm to return to the original configuration through the "get_path_to_configuration" function shared by the organizers.

In [3]:
# Prepare pixel travel map:
unvisited = np.ones([side,side]) # one = unvisited pixel; 0 = visited pixel
total = side*side - 1 #total number of pixels minus the origin
origin = [(64,0),(-32,0),(-16,0),(-8,0),(-4,0),(-2,0),(-1,0),(-1,0)] #origin configuration
config = origin.copy() #future configuration

result = [config]
pbar = tqdm(total=total)
# Continue until all locations have been visited:
while(total):
    
    # Optimization variables:
    cost = 1e6
    distance = 1e6
    found = False
    
    # Current configuration:
    base = get_position(config)
    base_arr = (base[0]+radius, base[1]+radius)
    unvisited[base_arr] = 0
    
    # Is the location one step below unvisited?
    if base[1]==-128: #if we reached the bottom border
        below = 0
    else:
        below = unvisited[(base_arr[0],base_arr[1]-1)]
    
    # Single-link step:
    for i in range(len(origin)): #for each arm link
        for d in [-1,1]: #for each direction
            # Rotate link and get new position and vertical displacement:
            config2 = rotate(config, i, d)
            pos = get_position(config2)
            dy = pos[1] - base[1]
            
            # Convert from cartesian to array coordinates and measure cost:
            pos_arr = (pos[0]+radius, pos[1]+radius)
            cost2 = 1 + color_cost(base_arr, pos_arr, image)
            
            # Must move down unless impossible:
            if unvisited[pos_arr] and cost2<cost and (dy<0 or (dy>=0 and below==0)): 
                config_next = config2.copy()
                cost = cost2
                found = True

    # Double-link step:
    for i in range(len(origin)-1):
        for d1 in [-1,1]:
            for j in range(i+1,len(origin)):
                for d2 in [-1,1]:
                    # Rotate two separate links, get position and vertical displacement:
                    config2 = rotate(config, i, d1)
                    config2 = rotate(config2, j, d2)
                    pos = get_position(config2)
                    dy = pos[1] - base[1]
                    
                    # Convert from cartesian to array coordinates and measure cost:
                    pos_arr = (pos[0]+radius, pos[1]+radius)
                    cost2 = np.sqrt(2) + color_cost(base_arr, pos_arr, image)
                    
                    # Must move down unless impossible:
                    if(unvisited[pos_arr] and cost2 < cost and below==0): 
                        config_next = config2.copy()
                        cost = cost2
                        found = True
                        
    # If an unvisited point was found, we are done for this step:
    if found:
        config = config_next.copy()
        pos = get_position(config)
        total -= 1
        pbar.update(1)
        result.append(config)
        
        
    # Otherwise, find the nearest unvisited point and go there ignoring the travel map:
    else:
        # Search every single pixel of the travel map for unvisited points:
        for i in range(side): 
            for j in range(side): 
                if unvisited[(i,j)]:
                    
                    # Measure the distance to the current point and choose the nearest one:
                    distance2 = np.sqrt((base_arr[0]-i)**2 + (base_arr[1]-j)**2)
                    if(distance2 < distance):
                        point = (i-radius, j-radius)
                        distance = distance2
                        
        # Go to the nearest unvisited point:
        path = get_path_to_point(config, point)[1:]
        
        # Output shortest trajectory:
        for config in path:
            pos = get_position(config)
            pos_arr = (pos[0]+radius, pos[1]+radius)
            
            # Update the travel map:
            if unvisited[pos_arr]:
                unvisited[pos_arr] = 0
                total -= 1
                pbar.update(1)
            
            result.append(config)
                
            base = pos

pbar.close()

# Return to origin:
path = get_path_to_configuration(config, origin)[1:]

result.extend(path)


  0%|          | 0/66048 [00:00<?, ?it/s]

## Post Processing

It turns out that we can remove points from our path that are visited more than once. Not sure yet how to include this in the visualisation.

In [4]:
def find_duplicate_points(path):
    
    duplicate_points = {}

    for c in path:

        p = get_position(c)

        if p != (0,0):
            duplicate_points[p] = duplicate_points.get(p, 0) + 1
            
    return duplicate_points
    
def vector_diff_one(path):
    
    for i in range(len(path) - 1):
        
        for c0, c1 in zip(path[i], path[i+1]):
            
            if abs(c0[0] - c1[0]) + abs(c0[1] - c1[1]) > 1:
#                 print(path[i])
#                 print(path[i+1])
                return False
            
    return True

def run_remove(path):
    
    print("-- run remove --")
    print(f"Current length: {len(path)}")
    
    duplicate_points = find_duplicate_points(path)

    i = len(path) - 2
    while i >= 0 :

        local_p = path[i:i+3]
        p = get_position(local_p[1])

        new_local_p = compress_path(local_p)

        if vector_diff_one(new_local_p) and duplicate_points.get(p, 0) > 1 and len(new_local_p) < 3:

            path = path[:i+1] + path[i+2:]
            duplicate_points[p] -= 1

        i -= 1


    print(f"New length: {len(path)}")
        
    return path

In [5]:
result = run_remove(result)
result = run_remove(result)


-- run remove --
Current length: 68189
New length: 67494
-- run remove --
Current length: 67494
New length: 67487


In [6]:
submission = pd.Series(
    [config_to_string(config) for config in result],
    name="configuration",
)

submission.head()

0     64 0;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
1    64 -1;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
2    64 -2;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
3    64 -3;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
4    64 -4;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
Name: configuration, dtype: object

In [7]:
submission.to_csv('submission.csv', index=False)